In [2]:
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics
import pandas as pd
import numpy as np
np.random.seed(0)
from sklearn.model_selection import GridSearchCV, StratifiedKFold  # , cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support, confusion_matrix  # , classification_report
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score
from scipy import integrate


In [3]:
DATA_PATH = '../data/features/'

tran_train = pd.read_pickle(DATA_PATH + "Ttrain_fixedOS.pickle")
tran_test = pd.read_pickle(DATA_PATH + "Ttest_fixedOS.pickle")
ntran = pd.read_pickle(DATA_PATH + "NT_fixedOS.pickle")

In [3]:
ntran.head()


,,Class,ObsCount,amplitude,beyond1st,flux_percentile_ratio_mid20,flux_percentile_ratio_mid35,flux_percentile_ratio_mid50,flux_percentile_ratio_mid65,flux_percentile_ratio_mid80,kurtosis,...,poly3_t3,poly4_t1,poly4_t2,poly4_t3,poly4_t4,skew,small_kurtosis,std,stetson_j,stetson_k
ID,copy_num,,,,,,,,,,,,,,,,,,,,,
CataID1121048016765,0,non-transient,37,1.325,0.270270,0.085689,0.139318,0.186693,0.273474,0.379936,0.806118,...,9.275549e-12,-0.000078,5.795073e-09,1.657338e-10,1.347946e-13,-0.121108,-3.267227,0.520336,-0.213986,0.658328
CataID1121048016959,0,non-transient,162,1.285,0.333333,0.121476,0.216808,0.321775,0.543173,0.720385,-0.216866,...,5.519138e-11,-0.000104,1.175388e-07,5.420264e-11,-1.443238e-15,-0.027053,-3.056722,0.497320,0.158238,0.711336
CataID1112021056668,0,non-transient,25,1.090,0.440000,0.074063,0.118072,0.167083,0.239397,0.649333,-0.578216,...,-2.461033e-10,0.000196,2.293719e-07,-2.428915e-10,3.115327e-14,-0.402302,-3.415020,0.617377,-0.326482,0.714699
CataID1112021056680,0,non-transient,342,0.770,0.201754,0.089792,0.222260,0.329158,0.509584,0.750055,4.943506,...,3.174950e-11,-0.000015,1.679038e-08,2.447224e-11,-1.943614e-14,1.654544,-3.026575,0.211430,0.096149,0.745172
CataID1112021056719,0,non-transient,368,0.210,0.271739,0.131166,0.262401,0.393776,0.525359,0.789431,1.958833,...,-1.345298e-12,0.000005,-3.588746e-08,2.733594e-12,1.793104e-14,-0.325084,-3.024680,0.049496,0.060285,0.769558


In [4]:
feats = np.array(ntran.columns[2:])
print(len(feats))
print(feats)

30
['amplitude' 'beyond1st' 'flux_percentile_ratio_mid20'
 'flux_percentile_ratio_mid35' 'flux_percentile_ratio_mid50'
 'flux_percentile_ratio_mid65' 'flux_percentile_ratio_mid80' 'kurtosis'
 'max_slope' 'median_absolute_deviation' 'median_buffer_range_percentage'
 'pair_slope_trend' 'pair_slope_trend_last_30' 'percent_amplitude'
 'percent_difference_flux_percentile' 'poly1_t1' 'poly2_t1' 'poly2_t2'
 'poly3_t1' 'poly3_t2' 'poly3_t3' 'poly4_t1' 'poly4_t2' 'poly4_t3'
 'poly4_t4' 'skew' 'small_kurtosis' 'std' 'stetson_j' 'stetson_k']


# Binary classification

In [7]:
tran_train['target'] = list(map(lambda x: 1 if x!='non-transient' else 0, tran_train.Class))
tran_test['target'] = list(map(lambda x: 1 if x!='non-transient' else 0, tran_test.Class))
ntran['target'] = list(map(lambda x: 1 if x!='non-transient' else 0, ntran.Class))

In [12]:
print(ntran.shape[0])
print(tran_train.shape[0])
print(tran_test.shape[0])

15193
35310
1059


In [13]:
nttrain,nttest = train_test_split(ntran,test_size=0.25)
all_train = pd.concat([tran_train,nttrain])
all_test = pd.concat([tran_test,nttest])

In [14]:
clf = RandomForestClassifier(n_estimators=50,random_state=0, class_weight='balanced')
clf.fit(all_train[feats], all_train.target)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [15]:
from sklearn.metrics import recall_score
print(recall_score(all_test.target,clf.predict(all_test[feats])))

0.8696883852691218


# 8-class classification


In [4]:
labels = ['SN', 'CV', 'AGN', 'HPM', 'Blazar', 'Flare']

ntranTrain,ntranTest = train_test_split(ntran,test_size=0.25)


tran_train.loc[~tran_train.Class.isin(labels), ['Class']] = 'Other'
tran_test.loc[~tran_test.Class.isin(labels), ['Class']] = 'Other'


In [5]:
all_train =pd.concat([ntranTrain,tran_train])
all_test =pd.concat([ntranTest,tran_test])

In [6]:
all_train.Class.value_counts()

non-transient    11394
SN               10956
Other             7755
CV                7007
AGN               3355
HPM               2552
Blazar            2046
Flare             1639
Name: Class, dtype: int64

## Balance the classes

In [27]:
g = all_train.groupby('Class')
g=g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True))
all_train= g

In [30]:
def manualFact(lab):
    labels = ['SN', 'CV', 'AGN', 'HPM', 'Blazar', 'Flare','Other','non-transient']
    return labels.index(lab)

In [35]:
all_train['target'] = list(map(lambda x: manualFact(x),all_train['Class']))
all_test['target'] = list(map(lambda x: manualFact(x),all_test['Class']))

In [37]:
feats = np.array(ntran.columns[2:])
print(len(feats))
print(feats)

30
['amplitude' 'beyond1st' 'flux_percentile_ratio_mid20'
 'flux_percentile_ratio_mid35' 'flux_percentile_ratio_mid50'
 'flux_percentile_ratio_mid65' 'flux_percentile_ratio_mid80' 'kurtosis'
 'max_slope' 'median_absolute_deviation' 'median_buffer_range_percentage'
 'pair_slope_trend' 'pair_slope_trend_last_30' 'percent_amplitude'
 'percent_difference_flux_percentile' 'poly1_t1' 'poly2_t1' 'poly2_t2'
 'poly3_t1' 'poly3_t2' 'poly3_t3' 'poly4_t1' 'poly4_t2' 'poly4_t3'
 'poly4_t4' 'skew' 'small_kurtosis' 'std' 'stetson_j' 'stetson_k']


In [1]:
def rf():
    params = {
        'n_estimators': [200, 700],
        'max_features': ['auto', 'sqrt', 'log2']
    }
    return RandomForestClassifier(random_state=0, class_weight='balanced'), params
def scorers():
    scoring = {'accuracy': make_scorer(accuracy_score),
               'precision': make_scorer(precision_score, average='weighted'),
               'recall': make_scorer(recall_score, average='weighted'),
               'f1_score': make_scorer(f1_score, average='weighted')
               }
    return scoring

In [2]:
model,params = rf()

grid_search = GridSearchCV(model, params, cv=StratifiedKFold(2), scoring=scorers(),
                               refit='f1_score', return_train_score=True,verbose=100)
grid_search.fit(all_train[feats], all_train.target)
# Copy classifier
clf = grid_search

NameError: name 'RandomForestClassifier' is not defined

In [43]:
scores = precision_recall_fscore_support(
        all_test.target, clf.predict(all_test[feats]), average='weighted')
print(scores)

(0.8411730571014187, 0.753190613421161, 0.7866815054637516, None)


In [44]:
pd.crosstab(clf.predict(all_test[feats]), all_test.target)

target,0,1,2,3,4,5,6,7
row_0,,,,,,,,
0,148,22,2,1,8,4,39,135
1,21,170,0,1,7,2,17,37
2,6,2,90,0,4,0,22,33
3,3,0,0,69,0,0,1,177
4,15,9,7,0,23,0,15,5
5,15,2,1,0,1,27,5,245
6,38,13,7,0,5,3,91,126
7,51,7,13,3,3,22,44,3041


In [ ]:
all_test.target